This file, once complete, will obtain a dataframe that contains every trade that occured during the 2022-23 NHL season. I am going to scrape the CapFriendly Website using beautiful soup and selenium and then use pandas to get it into the dataframe.

In [2]:
import requests
import pandas as pd
import time
import logging
import lxml
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

Now that necessary packages are installed, going to parse the needed url. This URL has a dynamic table that contains the cost per save of NHL Goalies for the 2022-23 season. So I am going to scrape the cap friendly website (Cost Per Save tab) and get this into a dataframe that I can eventually download and put into the MySQL database. As mentioned, since the table is dynamic and not static, the selenium package is needed because the table only expands to its full length once you scroll down.

In [3]:
# Set up the Selenium driver (make sure you have the correct path to your WebDriver)
driver_path = 'C:/STL Blues Analytics/SalaryCap/NHLSalaryCap/chromedriver.exe'  # Update this to your WebDriver path
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Open the webpage
url = "https://www.capfriendly.com/cost-per-save/2023/season"
driver.get(url)

# Get initial page height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for new content to load
    time.sleep(3)
    
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        break
    last_height = new_height

# Get the page source after all rows are loaded
page_source = driver.page_source
driver.quit()

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Find the table and convert it to a DataFrame
table = soup.find('table')
dfattempt = pd.read_html(str(table))[0]



C:\Users\nsofi\AppData\Local\Temp\ipykernel_14136\3267651250.py:36: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfattempt = pd.read_html(str(table))[0]


In [4]:
dfattempt.shape
dfattempt.tail(5)

,#,GOALIE,AGE,TYPE,EXPIRY,CAP HIT,P. BONUSES,GP,MIN,SH,SV,SV%,GAA,$/GP,$/MIN,$/SV
81,82,"Bobrovsky, Sergei",34,NaN,Stnd,UFA,"$10,000,000",50,2834,1466,1321.0,0.901,3.07,"$200,000","$3,528","$7,570"
82,83,"Brossoit, Laurent",29,NaN,Stnd,UFA,"$2,325,000",11,636,317,294.0,0.927,2.17,"$211,363","$3,655","$7,908"
83,84,"Varlamov, Semyon",34,NaN,Stnd,UFA,"$5,000,000",23,1334,689,629.0,0.913,2.70,"$217,391","$3,748","$7,949"
84,85,"Murray, Matt",28,NaN,Stnd,UFA,"$6,250,000",26,1473,761,687.0,0.903,3.01,"$240,384","$4,243","$9,097"
85,86,"Petersen, Cal",28,NaN,Stnd,UFA,"$5,000,000",10,543,257,223.0,0.868,3.76,"$500,000","$9,208","$22,421"


In [6]:
dfattempt.to_csv("CF Cost Per Save 2022-23.csv") 